In [1]:
import weave
import keras
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, ZeroPadding2D, MaxPool2D, Input, Flatten, Dense
from keras.models import Model

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
n_layers = 3
pad_size = 1
num_filters = 2
inputs = Input(shape=(3,32,32))
input_array = []

for _ in range(2**n_layers):
    x = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)
    x = Conv2D(num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)
    x = Conv2D(num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    x = MaxPool2D()(x)
    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)
    x = Conv2D(2*num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)
    x = Conv2D(2*num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    x = MaxPool2D()(x)
    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)
    x = Conv2D(4*num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)
    x = Conv2D(4*num_filters, kernel_size = (3,3), strides=(1,1), padding='valid')(x)
    input_array.append(x)



x = weave.pyrm_net(input_size=(8,8,4*num_filters),
                   n_layers = n_layers,
                   n_filters_start = 6*num_filters,
                   inputs = input_array,
                   n_gpus = 1,
                   min_dim = 4,
                   max_pool_loc = 3,
                   end_max_pool = False)

x = Flatten()(x)
x = Dense(100, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

GPU settings allow for inf layers
Minimum output size allow for 3 layers
Number of layers 3
First Layer Size: 4 (Number of Units)
('Disjoint:', True)
number of units 4
number of devices 8
number of inputs 8
First Layer Output Size: 4 (Number of Tensors)
('Disjoint:', True)
number of units 2
number of devices 8
number of inputs 4
('Disjoint:', True)
number of units 1
number of devices 8
number of inputs 2
Final Layer Size 1


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
print(y_test.shape)

(50000, 3, 32, 32)
(10000, 10)


In [5]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True)
model.fit(x_train[0:100], y_train[0:100], callbacks=[viewer])

Epoch 1/1
100/100 [==============================] - 5s 51ms/step - loss: 9.9666 - acc: 0.0900


In [6]:
model.count_params()

767366

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
zero_padding2d_49 (ZeroPadding2 (None, 3, 34, 34)    0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_55 (ZeroPadding2 (None, 3, 34, 34)    0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_61 (ZeroPadding2 (None, 3, 34, 34)    0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_paddi

In [8]:
model.fit(x_train, y_train, epochs = 10, validation_split=.02)

Train on 49000 samples, validate on 1000 samples
Epoch 1/10
49000/49000 [==============================] - 1399s 29ms/step - loss: 1.5560 - acc: 0.4428 - val_loss: 1.3635 - val_acc: 0.4940
Epoch 2/10
49000/49000 [==============================] - 1356s 28ms/step - loss: 1.2691 - acc: 0.5493 - val_loss: 1.2224 - val_acc: 0.5550
Epoch 3/10
49000/49000 [==============================] - 1354s 28ms/step - loss: 1.2017 - acc: 0.5749 - val_loss: 1.1520 - val_acc: 0.6000
Epoch 4/10
49000/49000 [==============================] - 1362s 28ms/step - loss: 1.1441 - acc: 0.5982 - val_loss: 1.1600 - val_acc: 0.5930
Epoch 5/10
49000/49000 [==============================] - 1347s 27ms/step - loss: 1.1071 - acc: 0.6114 - val_loss: 1.1227 - val_acc: 0.6000
Epoch 6/10
49000/49000 [==============================] - 1355s 28ms/step - loss: 1.0680 - acc: 0.6266 - val_loss: 1.1548 - val_acc: 0.6000
Epoch 7/10
49000/49000 [==============================] - 1353s 28ms/step - loss: 1.0576 - acc: 0.6323 - val_lo

KeyboardInterrupt: 

In [ ]:
model.evaluate()